<a href="https://colab.research.google.com/github/dlguswn3659/class-MachineLearning/blob/master/new_assignment_11_hidden_layer_four.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
nltk.download('wordnet')
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from google.colab import files

uploaded = files.upload()

import zipfile
import io
zf = zipfile.ZipFile(io.BytesIO(uploaded['movie_review.zip']), "r")
zf.extractall()

review_data = load_files(r"movie_review")
X, y = review_data.data, review_data.target

documents = []

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


Saving movie_review.zip to movie_review.zip


In [3]:
print(documents)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = CountVectorizer(max_features=40000, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
# vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(documents).toarray()
print(X)

vector_size = 2616

###################################################

# # tfidfconverter = TfidfTransformer()
# # print(tfidfconverter)
# # X = tfidfconverter.fit_transform(X).toarray()
# # print(X)

# X_train_tmp, X_test_tmp, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

# # print(X_train.shape)

# X_pos_sum = np.zeros(12638)
# X_neg_sum = np.zeros(12638)

# X_train = np.zeros((1401, 897))
# X_test = np.zeros((601, 897))

# for i in range(0, 2002):
#   if y[i] == 0:
#     X_neg_sum = X_neg_sum + X[i]
#   else:
#     X_pos_sum = X_pos_sum + X[i]

# pos_neg_voca = 0

# for i in range(0, 12638):
#   # if abs(X_neg_sum[i] - X_pos_sum[i]) >= 30:
#   if (X_neg_sum[i]*4 <= X_pos_sum[i] and X_pos_sum[i] > 8) or (X_pos_sum[i]*4 <= X_neg_sum[i] and X_neg_sum[i] > 8):
#     for j in range(0, 1401):
#       X_train[j][pos_neg_voca] = X_train_tmp[j][i]
#     for j in range(0, 601):
#       X_test[j][pos_neg_voca] = X_test_tmp[j][i]
#     pos_neg_voca  = pos_neg_voca + 1

# # tfidfconverter = TfidfTransformer()
# # X_train = tfidfconverter.fit_transform(X_train).toarray()
# # X_test = tfidfconverter.fit_transform(X_test).toarray()

# print(X_neg_sum)
# print(X_pos_sum)
# print(pos_neg_voca)

#################################################

list_x=X.tolist()
negative=[]
positive=[]
for i in range(0,2002):
  if y[i]==0:
    negative.append(list_x[i])
  else:
    positive.append(list_x[i])
negative=np.array(negative)
positive=np.array(positive)
total_sum=X.sum(axis=0)
neg_sum=negative.sum(axis=0)
pos_sum=positive.sum(axis=0)

neg_ratio=neg_sum/total_sum
pos_ratio=pos_sum/total_sum

good=[]
for i in range(0,len(pos_sum)):
  if neg_ratio[i]>pos_ratio[i]*3:
    good.append(i)
  elif pos_ratio[i]>neg_ratio[i]*3:
    good.append(i)
print("good의 길이:   " , len(good))
new=np.ones((1,2002), dtype=float)
for i in range(0,len(good)):
  temp=X[:,good[i]]
  temp=temp[np.newaxis]
  new=np.vstack((new,temp))
new=np.transpose(new)
new=new[:,1:]
# vectorizer = CountVectorizer(max_features=2, min_df=0, stop_words=stopwords.words('english'))
# Xp = vectorizer.fit_transform(pluss).toarray()

# tfidfconverter = TfidfTransformer()
# X = tfidfconverter.fit_transform(X).toarray()
X_train, X_test, y_train, y_test = train_test_split(new, y, test_size=0.3, shuffle=False)

print(X_train)
print(X_test)

print(X_train[0])

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
good의 길이:    2615
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[0. 0. 0. ... 0. 0. 0.]


In [5]:
print(X_train)
print(X_test)
print(y_train)
print(y_test)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[0 0 1 ... 1 0 1]
[1 1 0 1 1 1 1 0 0 0 0 1 1 0 0 1 1 1 0 0 0 0 1 1 1 0 0 1 1 1 0 0 0 1 1 0 1
 0 1 1 0 1 1 1 1 1 0 0 1 0 1 0 0 1 1 1 1 1 1 0 0 0 0 0 1 1 1 0 1 1 0 0 0 0
 0 0 1 1 1 0 1 1 1 0 1 1 0 1 1 0 1 1 0 0 0 1 0 0 0 1 1 1 0 0 1 1 1 1 1 0 0
 0 1 1 0 0 0 0 0 1 0 1 1 0 1 0 1 0 0 0 1 0 1 0 1 1 1 0 1 0 1 1 0 0 1 0 1 0
 1 1 1 1 0 0 0 0 1 1 1 1 1 0 0 1 1 1 1 0 1 1 0 0 1 0 1 1 1 0 1 0 0 1 1 1 0
 1 1 1 1 0 0 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 0 0 1
 1 1 1 0 1 0 0 0 1 1 0 1 1 0 0 0 1 1 1 0 0 0 1 0 0 1 0 0 1 1 1 1 1 1 1 0 0
 0 1 1 0 0 0 0 0 1 0 1 1 1 0 0 1 1 0 0 0 1 1 1 1 1 0 0 0 1 1 1 0 0 0 1 0 0
 0 0 0 1 1 1 0 0 1 1 0 0 1 1 1 0 0 0 0 0 1 0 0 1 1 0 1 1 1 0 0 1 1 1 1

In [0]:
#train_classifier에 대한 함수 정의하기
# def train_classifier(X_Train, y_Train):


In [0]:
# #Codes for classification

# classifier = train_classifier(X_train, y_train)
# y_pred_test = classifier.predict(X_test)
# y_pred_train = classifier.predict(X_train)

In [0]:
matrix_t = np.zeros((350, vector_size))  #350줄
matrix_t[0][0] = 1               #y레이어의 bias를 위한 값 1

for i in range(1, 350):
  for j in range(0, vector_size):
    matrix_t[i][j] = np.random.normal()


# matrix_u[196][0] = 0            #줄, 칸
##############################################
matrix_u = np.zeros((50, 350))  #50줄
matrix_u[0][0] = 1               #y레이어의 bias를 위한 값 1

for i in range(1, 50):
  for j in range(0, 350):
    matrix_u[i][j] = np.random.normal()


# matrix_u[196][0] = 0            #줄, 칸
##############################################
matrix_v = np.zeros((8, 50))  #8줄
matrix_v[0][0] = 1               #y레이어의 bias를 위한 값 1

for i in range(1, 8):
  for j in range(0, 50):
    matrix_v[i][j] = np.random.normal()


# matrix_v[196][0] = 0            #줄, 칸
##############################################
matrix_w = np.zeros((1, 8))  #1줄

for i in range(0, 1):
  for j in range(0, 8):
    matrix_w[i][j] = np.random.normal()

# matrix_w[196][0] = 0            #줄, 칸

# print(matrix)

In [9]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import matplotlib.animation as animation
import operator
from scipy import stats 
from mpl_toolkits.mplot3d import Axes3D
from sklearn.datasets.samples_generator import make_regression 
import csv
import math
import statistics

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
import operator

x_ = np.ones((350, 1))
y_ = np.ones((50, 1))

j_list = np.ones(shape=(1401, vector_size, 1))
x_list = np.ones(shape=(1401, 350, 1))

x__list = np.ones(shape=(1401, 350, 1))
y__list = np.ones(shape=(1401, 50, 1))
z__list = np.ones(shape=(1401, 8, 1))
h__list = np.ones(shape=(1401, 1, 1))

x_list = np.ones(shape=(1401, 350, 1))
y_list = np.ones(shape=(1401, 50, 1))
z_list = np.ones(shape=(1401, 8, 1))
h_list = np.ones(shape=(1401, 1, 1))
l_list = np.zeros(shape=(1401, 1, 1))

x_list_tmp = np.ones(shape=(1401, 350, 1))
y_list_tmp = np.ones(shape=(1401, 50, 1))
z_list_tmp = np.ones(shape=(1401, 8, 1))
h_list_tmp = np.ones(shape=(1401, 1, 1))

x__list_tmp = np.ones(shape=(1401, 350, 1))
y__list_tmp = np.ones(shape=(1401, 50, 1))
z__list_tmp = np.ones(shape=(1401, 8, 1))
h__list_tmp = np.ones(shape=(1401, 1, 1))


#training set 1401개에 대해서
for i in range(0, 1401):  
  bias   = [1]       #bias 1
  im_vector = np.concatenate((bias, X_train[i]), axis = None)
  j_list[i] = im_vector.reshape((vector_size, 1))

  x_ = np.dot(matrix_t, j_list[i])  #x와 가중치 u를 곱해서 y_를 만듦
  x__list[i] = x_
  
  x_vector = np.ones((350,1)) #bias
  for j in range (1, 350):
    x_vector[j][0] = 1 / (1 + math.exp(-x_[j][0]))
    # sigmoid(y_[j][0])  #sigmoid 함수를 적용해서 y_에서 y를 만듦

  x   = x_vector.reshape((350, 1))
  x_list[i] = x



  y_ = np.dot(matrix_u, x)  #x와 가중치 u를 곱해서 y_를 만듦
  y__list[i] = y_
  
  y_vector = np.ones((50,1)) #bias
  for j in range (1, 50):
    y_vector[j][0] = 1 / (1 + math.exp(-y_[j][0]))
    # sigmoid(y_[j][0])  #sigmoid 함수를 적용해서 y_에서 y를 만듦

  y   = y_vector.reshape((50, 1))
  y_list[i] = y

  z_  = np.dot(matrix_v, y)  #y와 가중치 v를 곱해서 z_를 만듦
  z__list[i] = z_
  z_vector = np.ones((8,1)) #bias
  for j in range (1, 8):
    z_vector[j][0] = 1 / (1 + math.exp(-z_[j][0]))  #sigmoid 함수를 적용해서 y_에서 y를 만듦

  z   = z_vector.reshape((8, 1))
  z_list[i] = z
  
  # print(z)

  h_  = np.dot(matrix_w, z)  #z와 가중치 w를 곱해서 h_를 만듦
  h__list[i] = h_
  
  h_vector = np.ones((1,1)) #bias
  for j in range (0, 1):
    h_vector[j][0] = 1 / (1 + math.exp(-h_[j][0]))  #sigmoid 함수를 적용해서 h_에서 h를 만듦

  h   = h_vector.reshape((1, 1))
  # tmp = h.ravel()
  # max_value = max(tmp)
  # print(max_value)
  # max_index = tmp.index(max_value)
  h_list[i] = h

  tmp_label = 0
  # print(h)

  if h >= 0.5:
    tmp_label = 1

  l_list[i][0] = tmp_label
  # print(l_list[i])
  # print(h)

  x_list_tmp[i] = x_list[i]
  y_list_tmp[i] = y_list[i]
  z_list_tmp[i] = z_list[i]
  h_list_tmp[i] = h_list[i]
  x__list_tmp[i] = x__list[i]
  y__list_tmp[i] = y__list[i]
  z__list_tmp[i] = z__list[i]
  h__list_tmp[i] = h__list[i]

# print(l_list)

In [0]:
import operator

x_2 = np.ones((350, 1))
y_2 = np.ones((50, 1))

j_list2 = np.ones(shape=(601, vector_size, 1))
x_list2 = np.ones(shape=(601, 350, 1))

x__list2 = np.ones(shape=(601, 350, 1))
y__list2 = np.ones(shape=(601, 50, 1))
z__list2 = np.ones(shape=(601, 8, 1))
h__list2 = np.ones(shape=(601, 1, 1))

x_list2 = np.ones(shape=(601, 350, 1))
y_list2 = np.ones(shape=(601, 50, 1))
z_list2 = np.ones(shape=(601, 8, 1))
h_list2 = np.ones(shape=(601, 1, 1))
l_list2 = np.zeros(shape=(601, 1, 1))

x_list_tmp2 = np.ones(shape=(601, 350, 1))
y_list_tmp2 = np.ones(shape=(601, 50, 1))
z_list_tmp2 = np.ones(shape=(601, 8, 1))
h_list_tmp2 = np.ones(shape=(601, 1, 1))

x__list_tmp2 = np.ones(shape=(601, 350, 1))
y__list_tmp2 = np.ones(shape=(601, 50, 1))
z__list_tmp2 = np.ones(shape=(601, 8, 1))
h__list_tmp2 = np.ones(shape=(601, 1, 1))


#training set 1401개에 대해서
for i in range(0, 601):  
  bias2   = [1]       #bias 1
  im_vector2 = np.concatenate((bias2, X_test[i]), axis = None)
  j_list2[i] = im_vector2.reshape((vector_size, 1))

  x_2 = np.dot(matrix_t, j_list2[i])  #x와 가중치 u를 곱해서 y_를 만듦
  x__list2[i] = x_2
  
  x_vector2 = np.ones((350,1)) #bias
  for j in range (1, 350):
    x_vector2[j][0] = 1 / (1 + math.exp(-x_2[j][0]))
    # sigmoid(y_[j][0])  #sigmoid 함수를 적용해서 y_에서 y를 만듦

  x2   = x_vector2.reshape((350, 1))
  x_list2[i] = x2



  y_2 = np.dot(matrix_u, x2)  #x와 가중치 u를 곱해서 y_를 만듦
  y__list2[i] = y_2
  
  y_vector2 = np.ones((50,1)) #bias
  for j in range (1, 50):
    y_vector2[j][0] = 1 / (1 + math.exp(-y_2[j][0]))
    # sigmoid(y_[j][0])  #sigmoid 함수를 적용해서 y_에서 y를 만듦

  y2   = y_vector2.reshape((50, 1))
  y_list2[i] = y2

  z_2  = np.dot(matrix_v, y2)  #y와 가중치 v를 곱해서 z_를 만듦
  z__list2[i] = z_2
  z_vector2 = np.ones((8,1)) #bias
  for j in range (1, 8):
    z_vector2[j][0] = 1 / (1 + math.exp(-z_2[j][0]))  #sigmoid 함수를 적용해서 y_에서 y를 만듦

  z2   = z_vector2.reshape((8, 1))
  z_list2[i] = z2
  
  # print(z)

  h_2 = np.dot(matrix_w, z2)  #z와 가중치 w를 곱해서 h_를 만듦
  h__list2[i] = h_2
  
  h_vector2 = np.ones((1,1)) #bias
  for j in range (0, 1):
    h_vector2[j][0] = 1 / (1 + math.exp(-h_2[j][0]))  #sigmoid 함수를 적용해서 h_에서 h를 만듦

  h2   = h_vector2.reshape((1, 1))
  # tmp = h.ravel()
  # max_value = max(tmp)
  # print(max_value)
  # max_index = tmp.index(max_value)
  h_list2[i] = h2

  tmp_label = 0

  if h2 >= 0.5:
    tmp_label = 1

  l_list2[i][0] = tmp_label
  # print(l_list[i])
  # print(h2)

  x_list_tmp2[i] = x_list2[i]
  y_list_tmp2[i] = y_list2[i]
  z_list_tmp2[i] = z_list2[i]
  h_list_tmp2[i] = h_list2[i]
  x__list_tmp2[i] = x__list2[i]
  y__list_tmp2[i] = y__list2[i]
  z__list_tmp2[i] = z__list2[i]
  h__list_tmp2[i] = h__list2[i]

# print(l_list)

In [12]:
# (1401, 1500)
# (601, 1500)
# (1401,)
# (601,)

iteration = 2000
l_r = 0.2
lambda_ = 1

cost_history = np.zeros(iteration)    #about train
cost_history2 = np.zeros(iteration)   #about test

label_group = np.zeros(shape = (1401, 1))
label_group2 = np.zeros(shape = (601, 1))

accuracy_history = np.zeros(iteration)
accuracy_history2 = np.zeros(iteration)

for i in range(0, 1401):
  label_group[i][0] = l_list[i][0]

for i in range(0, 601):
  label_group2[i][0] = l_list2[i][0]

for t in range(0, iteration):
  number_of_correct = 0
  number_of_correct2 = 0

  print(t, end=' ')

  parameter_sum = 0

  for q in range(0, vector_size):
    for w in range(1, 350):
      parameter_sum = parameter_sum + matrix_t[w][q]**2

  for q in range(0, 350):
    for w in range(1, 50):
      parameter_sum = parameter_sum + matrix_u[w][q]**2

  for q in range(0, 50):
    for w in range(1, 8):
      parameter_sum = parameter_sum + matrix_v[w][q]**2

  for q in range(0, 8):
    for w in range(0, 1):
      parameter_sum = parameter_sum + matrix_w[w][q]**2

  result = 0.0
  for i in range(0, 1401):
    for k in range(0, 1):
      result = result + ((-1)*label_group[i][k]*math.log(h_list[i][k][0]) - (1-label_group[i][k])*math.log(1-h_list[i][k][0])) + ((lambda_)/(2 * 328008)) * parameter_sum
  result = result / 1401
  cost_history[t] = result

##gradient descent
  result_w = 0
  result_v = 0
  result_u = 0
  result_t = 0

  for i in range(0, 1401):
    small_delta_2 = np.zeros(shape = (1, 1))
    for j in range(0, 1):
      small_delta_2[j][0] = h_list[i][j] - label_group[i][j]
    # print(h_list[i])
    # print(label_group[i])
    # small_delta_2 = h_list[i] - label_group[i] #여기 label_group 다시 봐야함.
    small_delta_1 = np.dot(np.transpose(matrix_w), small_delta_2)
    small_delta_0 = np.dot(np.transpose(matrix_v), small_delta_1)*(y_list_tmp[i]*(1 - y_list_tmp[i]))
    small_delta_ = np.dot(np.transpose(matrix_u), small_delta_0)*(x_list_tmp[i]*(1 - x_list_tmp[i])) #이거..괜찮은가..?

    # print(small_delta_2)
    result_w = result_w + np.dot(small_delta_2, np.transpose(z_list[i]))
    result_v = result_v + np.dot(small_delta_1, np.transpose(y_list[i]))
    result_u = result_u + np.dot(small_delta_0, np.transpose(x_list[i]))
    result_t = result_t + np.dot(small_delta_, np.transpose(j_list[i]))

  matrix_w = matrix_w - (result_w / 1401 *l_r)
  matrix_v = matrix_v - (result_v / 1401 *l_r)
  matrix_u = matrix_u - (result_u / 1401 *l_r)
  matrix_t = matrix_t - (result_t / 1401 *l_r)


####################################################################

  for k in range(0, 1401):
    bias   = [1]       #bias 1
    im_vector = np.concatenate((bias, X_train[k]), axis = None)
    j_list[k] = im_vector.reshape((vector_size, 1))

    x_ = np.dot(matrix_t, j_list[k])  #x와 가중치 u를 곱해서 y_를 만듦
    x__list[k] = x_
    
    x_vector = np.ones((350,1)) #bias
    for j in range (1, 350):
      x_vector[j][0] = 1 / (1 + math.exp(-x_[j][0]))
      # sigmoid(y_[j][0])  #sigmoid 함수를 적용해서 y_에서 y를 만듦

    x   = x_vector.reshape((350, 1))
    x_list[k] = x



    y_ = np.dot(matrix_u, x)  #x와 가중치 u를 곱해서 y_를 만듦
    y__list[k] = y_
    
    y_vector = np.ones((50,1)) #bias
    for j in range (1, 50):
      y_vector[j][0] = 1 / (1 + math.exp(-y_[j][0]))
      # sigmoid(y_[j][0])  #sigmoid 함수를 적용해서 y_에서 y를 만듦

    y   = y_vector.reshape((50, 1))
    y_list[k] = y

    z_  = np.dot(matrix_v, y)  #y와 가중치 v를 곱해서 z_를 만듦
    z__list[k] = z_
    z_vector = np.ones((8,1)) #bias
    for j in range (1, 8):
      z_vector[j][0] = 1 / (1 + math.exp(-z_[j][0]))  #sigmoid 함수를 적용해서 y_에서 y를 만듦

    z   = z_vector.reshape((8, 1))
    z_list[k] = z
    
    # print(z)

    h_  = np.dot(matrix_w, z)  #z와 가중치 w를 곱해서 h_를 만듦
    h__list[k] = h_
    
    h_vector = np.ones((1,1)) #bias
    for j in range (0, 1):
      h_vector[j][0] = 1 / (1 + math.exp(-h_[j][0]))  #sigmoid 함수를 적용해서 h_에서 h를 만듦

    h   = h_vector.reshape((1, 1))
    # tmp = h.ravel()
    # max_value = max(tmp)
    # print(max_value)
    # max_index = tmp.index(max_value)

    h_list11_tmp = h_list.reshape((1401))
    median_val = statistics.median(h_list11_tmp)

    h_list[k] = h

    tmp_label = 0

    # if h >= median_val:
    #     # print(h)
    #     tmp_label = 1

    if h >= 0.5:
      # print(h)
      tmp_label = 1

    l_list[k][0] = tmp_label
    # print(l_list[k])
    # print(h)

    x_list_tmp[k] = x_list[k]
    y_list_tmp[k] = y_list[k]
    z_list_tmp[k] = z_list[k]
    h_list_tmp[k] = h_list[k]
    x__list_tmp[k] = x__list[k]
    y__list_tmp[k] = y__list[k]
    z__list_tmp[k] = z__list[k]
    h__list_tmp[k] = h__list[k]

    if l_list[k][0] == y_train[k]:
      number_of_correct = number_of_correct + 1

  accuracy_history[t] = number_of_correct / 1401 * 100
  print(accuracy_history[t], end=' ')
  # accuracy_history[t] = number_of_correct_predictions/total_number_of_predictions*100
  # print(accuracy_history[t])

############################
  #test data!

  number_of_correct_predictions2 = 0
  
  # print(t, end=' ')

  result2 = 0.0
  for i in range(0, 601):
    for k in range(0, 1):
      # if h_list_tmp[i][k][0] <= 0:
      # print(h_list_tmp[i][k][0])
      result2 = result2 + ((-1)*label_group2[i][k]*math.log(h_list2[i][k][0]) - (1-label_group2[i][k])*math.log(1-h_list2[i][k][0])) + ((lambda_)/(2 * 328008)) * parameter_sum
  result2 = result2 / 601
  cost_history2[t] = result2


  for k in range(0, 601):
  # print(i)
    bias2   = [1]       #bias 1
    im_vector2 = np.concatenate((bias2, X_test[k]), axis = None)
    j_list2[k] = im_vector2.reshape((vector_size, 1))

    x_2 = np.dot(matrix_t, j_list2[k])  #x와 가중치 u를 곱해서 y_를 만듦
    x__list2[k] = x_2
    
    x_vector2 = np.ones((350,1)) #bias
    for j in range (1, 350):
      x_vector2[j][0] = 1 / (1 + math.exp(-x_2[j][0]))
      # sigmoid(y_[j][0])  #sigmoid 함수를 적용해서 y_에서 y를 만듦

    x2   = x_vector2.reshape((350, 1))
    x_list2[k] = x2



    y_2 = np.dot(matrix_u, x2)  #x와 가중치 u를 곱해서 y_를 만듦
    y__list2[k] = y_2
    
    y_vector2 = np.ones((50,1)) #bias
    for j in range (1, 50):
      y_vector2[j][0] = 1 / (1 + math.exp(-y_2[j][0]))
      # sigmoid(y_[j][0])  #sigmoid 함수를 적용해서 y_에서 y를 만듦

    y2   = y_vector2.reshape((50, 1))
    y_list2[k] = y2

    z_2  = np.dot(matrix_v, y2)  #y와 가중치 v를 곱해서 z_를 만듦
    z__list2[k] = z_2
    z_vector2 = np.ones((8,1)) #bias
    for j in range (1, 8):
      z_vector2[j][0] = 1 / (1 + math.exp(-z_2[j][0]))  #sigmoid 함수를 적용해서 y_에서 y를 만듦

    z2   = z_vector2.reshape((8, 1))
    z_list2[k] = z2
    
    # print(z)

    h_2 = np.dot(matrix_w, z2)  #z와 가중치 w를 곱해서 h_를 만듦
    h__list2[k] = h_2
    
    h_vector2 = np.ones((1,1)) #bias
    for j in range (0, 1):
      h_vector2[j][0] = 1 / (1 + math.exp(-h_2[j][0]))  #sigmoid 함수를 적용해서 h_에서 h를 만듦

    h2   = h_vector2.reshape((1, 1))
    # tmp = h.ravel()
    # max_value = max(tmp)
    # print(max_value)
    # max_index = tmp.index(max_value)

    h_list22_tmp = h_list2.reshape((601))
    median_val2 = statistics.median(h_list22_tmp)

    h_list2[k] = h2

    tmp_label = 0

    # if h2 >= median_val2:
    #   # print(h2)
    #   tmp_label = 1

    if h2 >= 0.5:
      # print(h2)
      tmp_label = 1

    l_list2[k][0] = tmp_label
    # print(l_list[k])
    # print(h2)

    x_list_tmp2[k] = x_list2[k]
    y_list_tmp2[k] = y_list2[k]
    z_list_tmp2[k] = z_list2[k]
    h_list_tmp2[k] = h_list2[k]
    x__list_tmp2[k] = x__list2[k]
    y__list_tmp2[k] = y__list2[k]
    z__list_tmp2[k] = z__list2[k]
    h__list_tmp2[k] = h__list2[k]

    if l_list2[k][0] == y_test[k]:
      number_of_correct2 = number_of_correct2 + 1

  accuracy_history2[t] = number_of_correct2 / 601 * 100
  print(accuracy_history2[t])

  # accuracy_history2[t] = number_of_correct_predictions2/total_number_of_predictions2*100
  t = t + 1

0 49.892933618843685 50.249584026622294
1 49.892933618843685 50.249584026622294
2 49.892933618843685 50.249584026622294
3 49.892933618843685 50.249584026622294
4 49.892933618843685 50.249584026622294
5 49.892933618843685 50.249584026622294
6 49.892933618843685 50.249584026622294
7 49.892933618843685 50.249584026622294
8 49.892933618843685 50.249584026622294
9 49.892933618843685 50.249584026622294
10 49.892933618843685 50.249584026622294
11 49.892933618843685 50.249584026622294
12 49.892933618843685 50.249584026622294
13 49.892933618843685 50.249584026622294
14 49.892933618843685 50.249584026622294
15 49.892933618843685 50.249584026622294
16 49.892933618843685 50.249584026622294
17 49.892933618843685 50.249584026622294
18 49.892933618843685 50.249584026622294
19 49.892933618843685 50.249584026622294
20 49.892933618843685 50.249584026622294
21 49.892933618843685 50.249584026622294
22 49.892933618843685 50.249584026622294
23 49.892933618843685 50.249584026622294
24 49.892933618843685 50.2

KeyboardInterrupt: ignored

In [0]:
#그 코드를 바탕으로 training loss plotting 하기.
real_iter = iteration

fig,ax = plt.subplots(figsize=(12,8))

cost_tmp = np.zeros(real_iter)

for i in range(0, real_iter):
  cost_tmp[i] = cost_history[i]


# print(cost_history[1])

ax.set_ylabel('J(Theta)')
ax.set_xlabel('Iterations')
_=ax.plot(range(real_iter),cost_tmp,'b.', color = 'blue')
_=ax.plot(range(real_iter),cost_history2,'b.', color = 'red')

In [0]:
#그 코드를 바탕으로 training loss plotting 하기.
fig,ax = plt.subplots(figsize=(12,8))

# print(cost_history[1])

ax.set_ylabel('accuracy')
ax.set_xlabel('Iterations')
_=ax.plot(range(iteration),accuracy_history,'b.', color = 'blue')
_=ax.plot(range(iteration),accuracy_history2,'b.', color = 'red')
# print(ac_tmp)

In [0]:
#Codes for Evaluation
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

l_list_tmp = l_list.reshape((1401))
l_list2_tmp = l_list2.reshape((601))

print(confusion_matrix(y_train,l_list_tmp))
print(classification_report(y_train,l_list_tmp))
print(accuracy_score(y_train, l_list_tmp))

print(confusion_matrix(y_test,l_list2_tmp))
print(classification_report(y_test,l_list2_tmp))
print(accuracy_score(y_test, l_list2_tmp))

In [0]:
class_1 = 0

for i in range(0, 601):
  if l_list2_tmp[i] == 1:
    class_1 = class_1 + 1

print(class_1)

In [0]:
print(y_train)
print(l_list_tmp)